In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import utils
import random

In [2]:
# hyperparameters
batch_iterations = 100
full_iterations = 100
batch_size = 1000
learning_rate = 0.01
reg_eta = 0.001

# dimensionalities
dim_lstm = 300
dim_word = 300
dim_aspect = 5
dim_sentence = 80
dim_polarity = 3

# setup utils object
isSample = False
u = utils.UTILS(batch_size, dim_sentence, dim_polarity, isSample)

In [3]:
# define tf placeholders
X = tf.placeholder(tf.int32, [None, dim_sentence])
y = tf.placeholder(tf.float32, [None, dim_polarity])
seqlen = tf.placeholder(tf.int32, [None])

In [4]:
# define tf variables
with tf.variable_scope('lstm_vars'):
    with tf.variable_scope('weights', reuse = tf.AUTO_REUSE):
        lstm_w = tf.get_variable(
            name = 'softmax_w',
            shape = [dim_lstm, dim_polarity],
            initializer = tf.random_normal_initializer(0, 0.03),
            regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
        )
    with tf.variable_scope('biases', reuse = tf.AUTO_REUSE):
        lstm_b = tf.get_variable(
            name = 'softmax_b',
            shape = [dim_polarity],
            initializer = tf.random_normal_initializer(0, 0.03),
            regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
        )

Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
# define lstm model
def dynamic_lstm(inputs, seqlen):
#     inputs = tf.nn.dropout(inputs, keep_prob=1.0)
    with tf.name_scope('lstm_model'):
        lstm_cell = tf.contrib.rnn.LSTMCell(dim_lstm)
        outputs, state = tf.nn.dynamic_rnn(
            lstm_cell,
            inputs = inputs,
            sequence_length = seqlen,
            dtype = tf.float32,
            scope = 'lstm'
        )
        size = tf.shape(outputs)[0]
        index = tf.range(0, size) * dim_sentence + seqlen - 1 # batch_size
        output = tf.gather(tf.reshape(outputs, [-1, dim_lstm]), index)  # batch_size * n_hidden
    predict = tf.matmul(output, lstm_w) + lstm_b # batch_size x dim_polarity
    return predict, outputs


In [6]:
# define operations
# tf.reset_default_graph()
pred = dynamic_lstm(tf.nn.embedding_lookup(u.gloveDict, X), seqlen)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

ValueError: Shapes must be equal rank, but are 2 and 3
	From merging shape 0 with other shapes. for 'softmax_cross_entropy_with_logits/logits' (op: 'Pack') with input shapes: [?,3], [?,80,300].

In [ ]:
# batch training
test_X, test_y, test_seqlen, _ = u.getTestData()
with tf.Session() as sess:
    sess.run(init)
    for i in range(iterations):
        batch_X, batch_y, batch_seqlen, _ = u.nextBatch(batch_size)
        sess.run(optimizer, feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen})
        if i > 0 and i % 100 == 0:
            loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen})
            print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
            loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen})
            print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

In [ ]:
# full dataset training
test_X, test_y, test_seqlen, _ = u.getData('test')
train_X, train_y, train_seqlen, _ = u.getData('train')
with tf.Session() as sess:
    sess.run(init)
    for i in range(iterations):
        sess.run(optimizer, feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen})
        if i > 0 and i % 4 == 0:
            loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen})
            print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
            loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen})
            print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

In [ ]:
_, test = dynamic_lstm(tf.nn.embedding_lookup(u.gloveDict, X), seqlen)

In [ ]:
test

In [ ]:
test1 = tf.reshape(test, [-1, dim_lstm])

In [ ]:
test1

In [ ]:
tf.gather(test1, [0,1,2])